<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
<br>汉化的库: <a href="https://github.com/GoatCsu/CN-LLMs-from-scratch.git">https://github.com/GoatCsu/CN-LLMs-from-scratch.git</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# 第 6 章：进行文本分类的微调

In [ ]:
from importlib.metadata import version

pkgs = ["matplotlib",
        "numpy",
        "tiktoken",
        "torch",
        "tensorflow", # For OpenAI's pretrained weights
        "pandas"      # Dataset loading
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")
#查看版本号Ω

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/chapter-overview.webp" width=500px>

In [ ]:
# 防止某些单元格执行两次

from IPython.core.magic import register_line_cell_magic

executed_cells = set()
#使用一个 set 数据结构来存储已经执行过的单元格标识符
@register_line_cell_magic
#注册了一个名为 run_once 的魔法命令
def run_once(line, cell):
    if line not in executed_cells:
        get_ipython().run_cell(cell)
        executed_cells.add(line)
    else:
        print(f"Cell '{line}' has already been executed.")

## 6.1 不同种类的微调

- 本章节没有代码

- 常见微调大语言模型的方法是指 指令微调( instructionfinetuning)和分类微调(classification finetuning)
- 如下是:指令微调，这也是下一章节所要讲的内容

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/instructions.webp" width=500px>

- 如果您具有机器学习背景，分类微调是您可能已经熟悉. 举个例子，分类微调类似于训练卷积网络来对手写数字进行分类
- 在分类微调中，模型可以输出一定量的类标签（例如，“spam”和“not spam”）
- 分类微调模型只能预测它在训练期间所熟知的类别（例如，“垃圾邮件”或“非垃圾邮件”），而指令微调模型通常可以执行许多任务
- 我们可以将分类微调模型视为非常专业的模型;在实践中，创建专业化模型比创建在许多不同任务上表现良好的通用模型要容易得多

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/spam-non-spam.webp" width=500px>

## 6.2 准备数据集

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/overview-1.webp" width=500px>

- 本节准备我们用于分类微调的数据集
- 我们使用由垃圾邮件和非垃圾邮件组成的数据集来微调 LLM 对其进行分类
- 首先，我们下载并解压缩数据集

In [ ]:
import urllib.request
import zipfile
import os
from pathlib import Path

url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"
#完成导入数据
def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download and extraction.")
        return

    # 下载文件
    with urllib.request.urlopen(url) as response:
        with open(zip_path, "wb") as out_file:
            out_file.write(response.read())

    # 解压
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)
    # Add .tsv file extension
    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)
    print(f"File downloaded and saved as {data_file_path}")

download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)

- 数据集保存为制表符分隔的文本文件，我们可以将其加载到 pandas DataFrame 中

In [ ]:
import pandas as pd

df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
df

- 当我们检查类分布时，我们看到数据包含 “ham” （即 “not spam”） 的频率比 “spam” 高得多

In [ ]:
print(df["Label"].value_counts())

- 为了简化处理，并且出于教学目的考虑，我们选择使用小规模数据集，这有助于更快地进行对大语言模型进行微调。因此，我们对数据集进行了欠采样，使每个类别包含 747 个实例。
- （除了欠采样之外，还有其他几种方法可以处理类平衡，但它们超出了大语言模型相关书籍的范围; 你可以在 imbalanced-learn 中找到示例和更多信息）[`imbalanced-learn` 用户指南](https://imbalanced-learn.org/stable/user_guide.html)

In [ ]:
%%run_once balance_df
#单元模型启动

def create_balanced_dataset(df):
    
    num_spam = df[df["Label"] == "spam"].shape[0]
    #算一下spam出现的次数
    ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
    #随机采样 “ham” 样本
    balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])
    #合并 “spam” 和采样后的 “ham”
    return balanced_df


balanced_df = create_balanced_dataset(df)
print(balanced_df["Label"].value_counts())

- 随后我们把标签 "ham" and "spam" 变成“1”和“0”:

In [ ]:
%%run_once label_mapping
balanced_df["Label"] = balanced_df["Label"].map({"ham": 0, "spam": 1})    

In [ ]:
balanced_df

- 现在自定义一个函数,用于把数据集随机划分为训练集、验证集、测试集

In [ ]:
def random_split(df, train_frac, validation_frac):
    # 把数据打乱
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    # 计算分割系数
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    # 切割文件夹
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)
# 测试集大小默认为 0.2，即剩余部分。

train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)
#DataFrame 随机划分为训练集、验证集和测试集并保存

## 6.3 创建数据加载器

- 由于文本消息长度随机,因此在批量化组合训练数据之前要做数据统一化,如下操作可供选择
  1. 数据截断至批次中的最短长度
  2. 用“空白”填充文本至最长长度

- 选择操作2,填充数据
- 为此,我们将`<|endoftext|>` 作为填充标注

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/pad-input-sequences.webp?123" width=500px>

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))
#编码

- 下面的 `SpamDataset` 类标识训练数据集中最长的序列，并将填充token添加到其他序列以匹配该序列长度

In [ ]:
import torch
from torch.utils.data import Dataset


class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)
        #先读入文本
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["Text"]
        ]
        #编码
        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            #比最大长度大的文本,进行裁剪
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        # 填充到最长长度
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]
    def __getitem__(self, index):
        #获取指定索引的数据样本。
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )
    def __len__(self):
        return len(self.data)
        #输出长度
    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length

In [ ]:
train_dataset = SpamDataset(
    csv_file="train.csv",
    max_length=None,
    tokenizer=tokenizer
)

print(train_dataset.max_length)

- 我们还将验证和测试集填充到最长的训练序列
- 在本次代码中,长于最长训练示例的验证和测试集,在 `SpamDatase` 代码中被 `encoded_text[:self.max_length]` 截断
- 此行为完全是可选的，如果我们在验证和测试集情况下都设置 `max_length=Non`，它也会很好地工作

In [ ]:
val_dataset = SpamDataset(
    csv_file="validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)
test_dataset = SpamDataset(
    csv_file="test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)
#padding到最长

- 接下来，我们使用数据集来实例化数据加载器，这与前几章中创建数据加载器类似

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/batch.webp" width=500px>

In [ ]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8

torch.manual_seed(123)
#初始化给大小跟“头”
# 设置种子确保可复现
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)
#对齐超参数,但是区别一下训练集跟验证的区别

- 作为验证步骤，我们遍历数据加载器，并确保每个批次包含 8 个训练样本，其中每个训练样本由 120 个 token 组成

In [ ]:
print("Train loader:")
for input_batch, target_batch in train_loader:
    pass
    #如果这个数据在训练集出现过,越过

print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions", target_batch.shape)

- 最后，让我们通过打印查看每个数据集的批次数

In [ ]:
print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

## 6.4 用预训练权重初始化模型

- 在本节中，我们将初始化我们在上一章中使用的预训练模型

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/overview-2.webp" width=500px>

In [ ]:
CHOOSE_MODEL = "gpt2-small (124M)"
INPUT_PROMPT = "Every effort moves"

BASE_CONFIG = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "drop_rate": 0.0,        # Dropout rate
    "qkv_bias": True         # Query-key-value bias
}
#先输入超参数
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}
#输入模型的参数
BASE_CONFIG.update(model_configs[CHOOSE_MODEL])

assert train_dataset.max_length <= BASE_CONFIG["context_length"], (
    f"Dataset length {train_dataset.max_length} exceeds model's context "
    f"length {BASE_CONFIG['context_length']}. Reinitialize data sets with "
    f"`max_length={BASE_CONFIG['context_length']}`"
)
# 断言语句，用于验证 train_dataset.max_length 是否符合模型的上下文长度（context_length）要求。
# 如果不符合，将抛出一个异常的讯号

In [ ]:
from gpt_download import download_and_load_gpt2
from previous_chapters import GPTModel, load_weights_into_gpt

model_size = CHOOSE_MODEL.split(" ")[-1].lstrip("(").rstrip(")")
settings, params = download_and_load_gpt2(model_size=model_size, models_dir="gpt2")
#引入超参数跟模型设置
model = GPTModel(BASE_CONFIG)
load_weights_into_gpt(model, params)
model.eval();

- 为了确保模型已正确加载，让我们仔细检查它是否生成了连贯的文本

In [ ]:
from previous_chapters import (
    generate_text_simple,
    text_to_token_ids,
    token_ids_to_text
)


text_1 = "Every effort moves you"

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_1, tokenizer),
    max_new_tokens=15,
    context_size=BASE_CONFIG["context_length"]
)
#经典的初始化
print(token_ids_to_text(token_ids, tokenizer))

- 在我们将模型微调为分类器之前，让我们看看该模型是否已经可以通过提示对垃圾邮件进行分类

In [ ]:
text_2 = (
    "Is the following text 'spam'? Answer with 'yes' or 'no':"
    " 'You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award.'"
)

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(text_2, tokenizer),
    max_new_tokens=23,
    context_size=BASE_CONFIG["context_length"]
)

print(token_ids_to_text(token_ids, tokenizer))

- 正如我们所看到的，该模型不太擅长遵循指示
- 这是意料之中的，因为它只是经过了预训练，没有经过指令微调（指令微调将在下一章中介绍）

## 6.5 添加分类探测头

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/lm-head.webp" width=500px>

- 在本节中，我们将修改预训练的 LLM，使其为分类微调做好准备
- 首先，让我们先看一下模型的架构

In [ ]:
print(model)

- 我们可以看到,在第 4 章中实现的架构
- 目标是替换和微调输出层
- 为了实现这一点，我们首先冻结模型，这意味着我们使所有层都是不可训练的

In [ ]:
for param in model.parameters():
    param.requires_grad = False
#返回 model 中所有的参数,但是不参与反向传播

- 然后，我们替换输出层`(model.out_head)`，它最初将层输入映射到 50,257 个维度（词汇表的大小）
- 由于我们对二元分类模型进行了微调（预测 2 个类别，“spam”和“not spam”），因此我们可以替换如下所示的输出层，默认情况下它是可训练的
- 我们使用 `BASE_CONFIG["emb_dim"]`（等于 “gpt2-small （124M）” 模型中的 768）来保持下面的代码更通用

In [ ]:
torch.manual_seed(123)

num_classes = 2
model.out_head = torch.nn.Linear(in_features=BASE_CONFIG["emb_dim"], out_features=num_classes)

- 从技术上讲，只需训练输出层就足够了
- 但是，正如在 [Finetuning Large Language Models](https://magazine.sebastianraschka.com/p/finetuning-large-language-models)中表明，微调其他层可以显著提高性能
- 因此，我们还使最后一个 transformer 模块和最后一个 `LayerNorm` 模块连接起来，将最后一个 transformer 模块连接到输出层，使其可训练

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/trainable.webp" width=500px>

In [ ]:
for param in model.trf_blocks[-1].parameters():
    param.requires_grad = True

for param in model.final_norm.parameters():
    param.requires_grad = True
#但是对于要更新权重的就需要引入权重了

- 这个模型仍然适用
- 例如,我们输入点文本

In [ ]:
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape)
#编码并输出

- 与前几章相比，它现在有两个输出维度，而不是 50,257 个

In [ ]:
with torch.no_grad():
    outputs = model(inputs)

print("Outputs:\n", outputs)
print("Outputs dimensions:", outputs.shape) # shape: (batch_size, num_tokens, num_classes)

- 如前几章所述，对于每个 input token，都有一个 output vector
- 由于我们向模型提供了具有 4 个输入 token 的文本样本，因此输出由上面的 4 个 2 维输出向量组成

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/input-and-output.webp" width=500px>

- 在第 3 章中，我们讨论了attention机制，它将每个输入 token 连接到另一个输入 token
- 在第 3 章中，我们还介绍了类 GPT 模型中使用的因果注意力掩码(causal attention mask);这种掩码让当前token只关注当前和先前出现过的位置
- 基于这种因果注意力机制，第 4 个（最后一个）token在所有token中包含最多的信息，因为唯一包含所有其他全部信息只有它
- 因此，我们对最后一个 token 特别感兴趣，我们将针对垃圾邮件分类任务对其进行微调

In [ ]:
print("Last output token:", outputs[:, -1, :])

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/attention-mask.webp" width=200px>

## 6.6 计算分类损失以及分类准确性

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/overview-3.webp?1" width=500px>

- 在解释损失计算之前，让我们简单了解一下模型输出是如何转换为类标签的

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/class-argmax.webp" width=600px>

In [ ]:
print("Last output token:", outputs[:, -1, :])

- 与第 5 章类似，我们通过 `softmax` 函数将输出 （logits） 转换为概率分数，然后通过`argmax`函数获得最大概率值的索引位置

In [ ]:
probas = torch.softmax(outputs[:, -1, :], dim=-1)
label = torch.argmax(probas)
print("Class label:", label.item())

- 如第 5 章所述，softmax 函数在这里是可选的，因为最大的输出对应于最大的概率分数

In [ ]:
logits = outputs[:, -1, :]
label = torch.argmax(logits)
print("Class label:", label.item())

- 我们可以应用这个概念来计算所谓的"分类准确性"，计算给定数据集中正确预测的百分比
- 为了计算分类准确率，我们可以将前面基于 `argmax` 的预测代码应用于数据集中的所有示例，并按如下方式计算正确预测的分数：

In [ ]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model.eval()
    correct_predictions, num_examples = 0, 0
    #先改成评价模式,在初始化两个储存器
    if num_batches is None:
        num_batches = len(data_loader)
        #末了,给出了长度
    else:
        num_batches = min(num_batches, len(data_loader))
        #取短板
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)
            #仅处理前 num_batches 批次的数据
            with torch.no_grad():
                logits = model(input_batch)[:, -1, :]  # Logits of last output token
            predicted_labels = torch.argmax(logits, dim=-1)
            #预测内容取最大值
            num_examples += predicted_labels.shape[0]
            #当前批次的样本数
            correct_predictions += (predicted_labels == target_batch).sum().item()
            #预测正确的样本数量
        else:
            break
    return correct_predictions / num_examples

- 用该函数计算不同数据集的分类精度

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Note:

#取消注释以下行将允许代码在 Apple Silicon 芯片上运行（如果适用），
#这比在 Apple CPU 上运行快大约 2 倍（根据 M3 MacBook Air 的测量结果）。
#截至目前，在 PyTorch 2.4 版本中，通过 CPU 和 MPS 得到的结果是相同的。
#然而，在 PyTorch 的早期版本中，使用 MPS 时，可能会观察到不同的结果。
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Running on {device} device.")

model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes

torch.manual_seed(123) # For reproducibility due to the shuffling in the training data loader

train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=10)
val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=10)
test_accuracy = calc_accuracy_loader(test_loader, model, device, num_batches=10)
#各种计算
print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")

- 可以看到，预测的准确性不是很好，这是因为因为我们还没有对模型进行微调

- 在开始微调 （/training） 之前，我们首先必须定义要在训练期间优化的损失函数
- 目标是最大限度地提高模型的垃圾邮件分类准确性;但是，分类准确性不是一个可微分的函数
- 因此，相反，我们将交叉熵损失最小化，作为最大化分类精度的代理（您可以在我免费提供的[深度学习入门课程](https://sebastianraschka.com/blog/2021/dl-course.html#l08-multinomial-logistic-regression--softmax-regression)的第 8 讲中了解有关此主题的更多信息）

- `calc_loss_batch` 函数与第 5 章相同，只是我们只对优化最后一个 `tokens model（input_batch）``[：， -1， ：]` 感兴趣，而不是所有 `tokens model（input_batch）`

In [ ]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)[:, -1, :]  # Logits of last output token
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss
    #用交叉熵计算损失函数

使用 `calc_closs_loader`，我们在开始训练之前计算初始训练、验证和测试集损失

In [ ]:

def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        #如果 num_batches 超过数据加载器中的批次数，则减少批次数以匹配数据加载器中的总批次数
        #num_batches = min(num_batches, len(data_loader))
        for i, (input_batch, target_batch) in enumerate(data_loader):
            if i < num_batches:
                loss = calc_loss_batch(input_batch, target_batch, model, device)
                total_loss += loss.item()
            #总损失值
            else:
                break
    return total_loss / num_batches
    #

- 在下一节中，我们将训练模型以提高损失值，从而提高分类准确性

In [ ]:
with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device, num_batches=5)
    val_loss = calc_loss_loader(val_loader, model, device, num_batches=5)
    test_loss = calc_loss_loader(test_loader, model, device, num_batches=5)

print(f"Training loss: {train_loss:.3f}")
print(f"Validation loss: {val_loss:.3f}")
print(f"Test loss: {test_loss:.3f}")

- 在下一节中，我们将训练模型以提高损失值，从而提高分类准确性

## 6.7 有监督训练微调

- 在本节中，我们将定义并使用训练函数来提高模型的分类准确率
- 下面的 `train_classifier_simple` 函数实际上与我们在第 5 章中用于预训练模型的 `train_model_simple `函数相同
- 唯二区别
  1. 跟踪看到的训练样本数 （examples_seen），而不是看到的 token 数
  2. 计算每个 epoch 后的准确性，而不是在每个 epoch 后打印示例文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/training-loop.webp?1" width=500px>

In [ ]:
#跟第五章的一摸一样那我就当复习了
def train_classifier_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                            eval_freq, eval_iter):
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    examples_seen, global_step = 0, -1
    #初始化分类头
    for epoch in range(num_epochs):
        model.train()  
        #每次都进入训练模块
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() 
            #清梯度
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() 
            #对损失值执行 反向传播记录梯度
            optimizer.step() 
            #用梯度优化权重
            examples_seen += input_batch.shape[0] 
            global_step += 1

            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
        val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=eval_iter)
        print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
        print(f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)

    return train_losses, val_losses, train_accs, val_accs, examples_seen

- `evaluate_model` 在`train_classifier_simple` 是跟第五章相同的

In [ ]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss
    #调用辅助函数 calc_loss_loader 计算模型在 训练集、验证上的损失

- M3 MacBook Air五分钟训练完
- V100 or A100 GPU大概用半分钟

In [ ]:
import time

start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay=0.1)

num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")
#输出每一次的损失值跟在一定频次下进行准确性输出

- 跟第五章相似,我们用Matplot作图

In [ ]:
import matplotlib.pyplot as plt

def plot_values(epochs_seen, examples_seen, train_values, val_values, label="loss"):
    fig, ax1 = plt.subplots(figsize=(5, 3))

    ax1.plot(epochs_seen, train_values, label=f"Training {label}")
    ax1.plot(epochs_seen, val_values, linestyle="-.", label=f"Validation {label}")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel(label.capitalize())
    ax1.legend()

   
    ax2 = ax1.twiny()  
    ax2.plot(examples_seen, train_values, alpha=0)  

    fig.tight_layout()  
    plt.savefig(f"{label}-plot.pdf")
    plt.show()
#一个经典的画图操作

In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_losses))

plot_values(epochs_tensor, examples_seen_tensor, train_losses, val_losses)

- 图中, 根据训练的斜率, 我们可以发现模型训练的很好
- 此外, 训练和验证损失非常接近可以表明，该模型不会倾向于过拟合训练数据
- 同样的, 我们可以对精度进行绘制

In [ ]:
epochs_tensor = torch.linspace(0, num_epochs, len(train_accs))
examples_seen_tensor = torch.linspace(0, examples_seen, len(train_accs))

plot_values(epochs_tensor, examples_seen_tensor, train_accs, val_accs, label="accuracy")

- 根据上面的准确率图，我们可以看到该模型在第 4 个和第 5 循环之后,训练和验证准确率变得相对较高
- 但是，别忘了我们之前在 training 函数中指定了 `eval_iter=5`，这意味着我们只估计了训练和验证集的性能
- 我们可以计算完整数据集的训练、验证和测试集性能，如下所示

In [ ]:
train_accuracy = calc_accuracy_loader(train_loader, model, device)
val_accuracy = calc_accuracy_loader(val_loader, model, device)
test_accuracy = calc_accuracy_loader(test_loader, model, device)

print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")
#输出总的值

- 我们可以看到，训练集和验证集的表现实际上是相同的。
- 然而，由于测试集表现略微较差，我们可以看出，模型在一定程度上对训练数据进行了过拟合，同时也对用于调整超参数（如学习率）的验证数据进行了过拟合。
- 不过，这种情况是正常的，并且通过增加模型的 dropout rate（drop_rate）或优化器设置中的 weight_decay，可能进一步减小这种差距。

## 6.8 用LLM做垃圾邮件检测器

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch06_compressed/overview-4.webp" width=500px>

- 最后，让我们将微调后的 GPT 模型投入实际应用。
- 以下的 `classify_review` 函数实现了类似于我们之前实现的 `SpamDataset` 的数据预处理步骤。
- 然后，函数返回模型预测的整数类别标签，并返回对应的类别名称。

In [ ]:
def classify_review(text, model, tokenizer, device, max_length=None, pad_token_id=50256):
    model.eval()
    # 将模型设置为评估模式，禁用dropout等训练特定操作。
    input_ids = tokenizer.encode(text)
    # 使用tokenizer将输入文本 转换token ID 列表。
    supported_context_length = model.pos_emb.weight.shape[0]
    # 获取模型支持的最大上下文长度（由位置嵌入的权重数量决定）。
    input_ids = input_ids[:min(max_length, supported_context_length)]
    # 如果 token 序列长度超过 或模型支持的最大长度进行截断。
    input_ids += [pad_token_id] * (max_length - len(input_ids))
    # 对序列进行填充
    input_tensor = torch.tensor(input_ids, device=device).unsqueeze(0) 
    # 将 token ID 转换为张量并移动到指定设备（如 GPU），同时增加 batch 维度。
    # Model inference
    with torch.no_grad():
        logits = model(input_tensor)[:, -1, :] 
        # 禁用梯度计算，仅进行推理。
        # 将输入张量传入模型，获取 logits，并提取最后一个 token 的 logits。
    predicted_label = torch.argmax(logits, dim=-1).item()
    # 在 logits 上取 softmax 概率最大值的索引，作为预测类别。
    return "spam" if predicted_label == 1 else "not spam"
    # 如果预测标签是 `1`，返回 "spam"，否则返回 "not spam"。

- 试一下

In [ ]:
text_1 = (
    "You are a winner you have been specially"
    " selected to receive $1000 cash or a $2000 award."
)

print(classify_review(
    text_1, model, tokenizer, device, max_length=train_dataset.max_length
))

In [ ]:
text_2 = (
    "Hey, just wanted to check if we're still on"
    " for dinner tonight? Let me know!"
)

print(classify_review(
    text_2, model, tokenizer, device, max_length=train_dataset.max_length
))

- 最后，让我们保存模型，以便以后如果需要重用模型时，无需重新训练。

In [ ]:
torch.save(model.state_dict(), "review_classifier.pth")
#储存!大功告成!

- 下次我们可以这么唤醒这个模型

In [ ]:
model_state_dict = torch.load("review_classifier.pth", map_location=device, weights_only=True)
model.load_state_dict(model_state_dict)

## 总结

- 请参阅 [./gpt_class_finetune.py](./gpt_class_finetune.py) 脚本，这是一个自包含的分类微调脚本。
- 你可以在 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 中找到练习解决方案。
- 此外，感兴趣的读者可以在 [附录E](../../appendix-E) 中找到关于低秩适应（LoRA）的参数高效训练介绍。